In [3]:
import pandas as pd



In [4]:
df=pd.read_csv("../sample_data/data1.csv")

In [5]:
df

,id,first_name,last_name,email,gender,phone,Channel,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Region
0,1,Chas,Rollett,crollett0@icq.com,Male,625-741-0213,2,12669,9656,7561,214,2674,1338,3
1,2,Rockie,Cheney,rcheney1@delicious.com,Male,233-712-5425,2,7057,9810,9568,1762,3293,1776,3
2,3,Gretel,Pole,gpole2@elpais.com,Female,514-912-5355,2,6353,8808,7684,2405,3516,7844,3
3,4,Bertine,Scogings,bscogings3@bizjournals.com,Female,622-446-1554,1,13265,1196,4221,6404,507,1788,3
4,5,Leonid,Shortcliffe,lshortcliffe4@ted.com,Male,384-201-2900,2,22615,5410,7198,3915,1777,5185,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,436,Valery,Ternott,vternottc3@bandcamp.com,Female,238-620-6990,1,29703,12051,16027,13135,182,2204,3
436,437,Roxy,Forsyde,rforsydec4@nymag.com,Female,945-246-6828,1,39228,1431,764,4510,93,2346,3
437,438,Jess,Skrines,jskrinesc5@wikia.com,Genderfluid,936-756-5245,2,14531,15488,30243,437,14841,1867,3
438,439,Burke,Sallan,bsallanc6@wp.com,Male,680-783-4295,1,10290,1981,2232,1038,168,2125,3


In [6]:
local_dataset=pd.DataFrame(columns=["Name","Contact","Email","Info","location","Product_to_pitch"])

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("InferSchema").getOrCreate()

# Automatically infers the schema while reading a CSV
df = spark.read.option("header", "true").csv("../sample_data/data1.csv")

# Show the inferred schema
df.printSchema()


root
 |-- id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- Channel: string (nullable = true)
 |-- Fresh: string (nullable = true)
 |-- Milk: string (nullable = true)
 |-- Grocery: string (nullable = true)
 |-- Frozen: string (nullable = true)
 |-- Detergents_Paper: string (nullable = true)
 |-- Delicassen: string (nullable = true)
 |-- Region: string (nullable = true)

